# Data Profiling

In [4]:
import pandas as pd

Загрузим данные из csv файла в датафрейм и выведем первые 5 строк

In [5]:
df = pd.read_csv('cf_test_dataset.csv', sep='\t')
df.head()

,user_id,server_order_id,service_id,server_configuration,service_start_date,service_end_date,user_id.1,user_name,user_surname,price
0,Userid1,8837,39,ServerConfig39,19.09.2018,19.10.2018,Userid1,UserName1,UserSurname1,1000.0
1,Userid1,8838,39,ServerConfig39,20.09.2018,20.10.2018,Userid1,UserName1,UserSurname1,1000.0
2,Userid1,15142,39,ServerConfig39,28.09.2018,28.10.2018,Userid1,UserName1,UserSurname1,1000.0
3,Userid1,15143,39,ServerConfig39,28.09.2018,28.10.2018,Userid1,UserName1,UserSurname1,1000.0
4,Userid1,16467,39,ServerConfig39,24.09.2018,24.10.2018,Userid1,UserName1,UserSurname1,1000.0


Проверим количество строк, типы данных столбцов и количество пустых значений

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               715 non-null    object 
 1   server_order_id       715 non-null    int64  
 2   service_id            715 non-null    int64  
 3   server_configuration  715 non-null    object 
 4   service_start_date    715 non-null    object 
 5   service_end_date      715 non-null    object 
 6   user_id.1             715 non-null    object 
 7   user_name             715 non-null    object 
 8   user_surname          715 non-null    object 
 9   price                 703 non-null    float64
dtypes: float64(1), int64(2), object(7)
memory usage: 56.0+ KB


По данному выводу можно сказать следующее:
- На первый взгляд, у нас два одинаковых столбца *user_id*. Проверим значения в этих столбцах ниже.
- Столбцы *service_start_date* и *service_end_date* нужно преобразовать в datetime
- Всего в файле 715 строк
- В 12 заказах нет цены

In [7]:
(df['user_id'] == df['user_id.1']).all()

True

Значения в столбцах одинаковые, один из столбцов можно удалить

In [8]:
df.drop(columns=['user_id.1'], inplace=True)

Посмотрим на заказы без цены:

In [10]:
df[df.price.isnull()]

,user_id,server_order_id,service_id,server_configuration,service_start_date,service_end_date,user_name,user_surname,price
60,Userid13,7144,24,ServerConfig24,07.09.2018,07.10.2018,UserName13,UserSurname13,NaN
61,Userid13,7145,24,ServerConfig24,07.09.2018,07.10.2018,UserName13,UserSurname13,NaN
62,Userid13,7146,24,ServerConfig24,07.09.2018,07.10.2018,UserName13,UserSurname13,NaN
483,Userid90,13134,39,ServerConfig39,28.09.2018,28.10.2018,UserName90,UserSurname90,NaN
484,Userid90,13135,39,ServerConfig39,28.09.2018,28.10.2018,UserName90,UserSurname90,NaN
485,Userid90,26532,36,ServerConfig36,03.09.2018,03.10.2018,UserName90,UserSurname90,NaN
686,Userid156,33473,9,ServerConfig9,10.09.2018,10.10.2018,UserName156,UserSurname156,NaN
697,Userid165,16995,19,ServerConfig19,02.10.2018,02.11.2018,UserName165,UserSurname165,NaN
698,Userid165,23869,6,ServerConfig6,28.09.2018,28.10.2018,UserName165,UserSurname165,NaN
699,Userid165,27582,6,ServerConfig6,20.09.2018,20.10.2018,UserName165,UserSurname165,NaN


Не считая цены, абсолютно нормальные заказы. Нужно связаться с владельцами данных и выяснить причину

Преобразуем нужные столбцы в datetime

In [11]:
df['service_start_date'] = pd.to_datetime(df['service_start_date'], format='%d.%m.%Y')

In [12]:
df['service_end_date'] = pd.to_datetime(df['service_end_date'], format='%d.%m.%Y')

Проверим наличие дубликатов в исходном датасете:

In [13]:
df.duplicated().sum()

0

Дубликатов нет

Посмотрим на распределение цены, нет ли выбросов

In [14]:
df['price'].describe()

count     703.000000
mean     2083.926031
std       738.104491
min      1000.000000
25%      1000.000000
50%      2400.000000
75%      2600.000000
max      3000.000000
Name: price, dtype: float64

Обнаруженные особенности в данных следует учесть при построении ETL пайплайна